In [1]:
# Работа с сервером
import requests
from pprint import pprint
import json

# Распаковка и парсинг
import zipfile
import pandas as pd
import io
from bs4 import BeautifulSoup
from datetime import datetime


from urllib.parse import urljoin, urlparse
import os
import shutil
import base64
import importlib
import re

from pathlib import Path

In [12]:
# Загружаем словари для отправки Мain info
from functions import get_dict

ranks = get_dict('ranks')
gender = get_dict('gender')
marital_statuses = get_dict('marital_statuses')
resident_statuses = get_dict('resident_statuses')
languages = get_dict('languages')


In [42]:
resident_statuses

[]

In [3]:
PATH = 'out/out_min'

In [4]:
# Все HTML файлы из папки
html_files = [str(p) for p in Path(PATH).rglob('*.html')]


In [105]:

import functions
importlib.reload(functions)
# обрабатываем все файлы из папки
for file in html_files:
    # получаем объект soup
    soup = functions.get_html_content(file)
    # парсим всю инфу
    main_info = functions.main_parser(soup)
    # и заметки
    notes = functions.parse_notes(soup)
    
    # собираем сырую инфу для подготовки POST запроса main
    # short_keys = [
        #  'Name / Surname:',
        # 'Position applied for:',
        #'Birthday / Place of birth:',
        # 'Country of residence / City:',
        # 'Citizenship:',
        # 'Phones:',
        # 'E-mail:'
        # ]
    # short_info = {k: main_info['Main info'][k] for k in short_keys}
    # genders = 
    personal_id =[]
    [personal_id.append(dct['No.']) 
     if dct['Title of document']=='International passport' 
     else None 
     for dct in main_info['Passports / Smbk']]
    # personal_id[0]
    # other_keys = [
    #     'Sex:',
    #     'Next of kin:',
        
    # ]
    # other_info = {k: main_info['Additional info'][k] for k in other_keys}  
    # # смотрим последний рейс
    # last_voyage = main_info['Sea service (last 5 years)'][0] if main_info['Sea service (last 5 years)'] else None
    # is_photo = f'Фото {"Есть" if functions.get_photo_simple(soup) else "Нет"}'
    # вывод резюме
    # pprint(file)
    # print(is_photo)
    # pprint(short_info, sort_dicts=False)
    # pprint(other_info, sort_dicts=False)
    # pprint(last_voyage)
    # pprint(notes)
    # print()
    print(personal_id[0] if functions.only_letters_digits_spaces(personal_id[0]) else None)
    print(main_info['Passports / Smbk'])
    

None
[{'Title of document': "Seaman's book", 'No.': '', 'Date of issue': '', 'Country of issue': '', 'Valid up': ''}, {'Title of document': 'International passport', 'No.': '', 'Date of issue': '', 'Country of issue': '', 'Valid up': ''}]
A16081502
[{'Title of document': "Seaman's book", 'No.': '195390', 'Date of issue': '09.07.2015', 'Country of issue': 'egypt', 'Valid up': '30.01.2020'}, {'Title of document': 'International passport', 'No.': 'A16081502', 'Date of issue': '02.09.2015', 'Country of issue': 'egypt', 'Valid up': '01.09.2022'}]
None
[{'Title of document': "Seaman's book", 'No.': '******', 'Date of issue': '19.02.2020', 'Country of issue': 'Russia', 'Valid up': '18.02.2025'}, {'Title of document': 'International passport', 'No.': '******', 'Date of issue': '26.03.2020', 'Country of issue': 'Russia', 'Valid up': '26.03.2025'}]
LV5660381
[{'Title of document': "Seaman's book", 'No.': "'0266410", 'Date of issue': '17.03.2017', 'Country of issue': 'Latvia', 'Valid up': '17.03.

In [106]:
main_info['Passports / Smbk']

[{'Title of document': "Seaman's book",
  'No.': 'BMA-SRB-17896-24',
  'Date of issue': '19.04.2024',
  'Country of issue': 'Bahamas',
  'Valid up': '19.04.2034'},
 {'Title of document': 'International passport',
  'No.': 'FJ 431255',
  'Date of issue': '24.10.2017',
  'Country of issue': 'Ukraine',
  'Valid up': '24.10.2027'}]

In [78]:
importlib.reload(functions)
# [dct['key'] for dct in 
functions.get_dicts_list(False)
#  ]


{'dictionaries': [{'is_visible': True,
   'key': 'vessel_propulsion_types',
   'name': 'Propulsion types',
   'ui_group_id': 6},
  {'is_visible': True,
   'key': 'vessel_certificate_groups',
   'name': 'Vessel Certificate Groups',
   'ui_group_id': 4},
  {'is_visible': True,
   'key': 'seafarer_relation_types',
   'name': 'Relation types',
   'ui_group_id': 5},
  {'is_visible': True,
   'key': 'languages',
   'name': 'Languages',
   'ui_group_id': 5},
  {'is_visible': True,
   'key': 'vessel_types',
   'name': 'Vessels types',
   'ui_group_id': 0},
  {'is_visible': True,
   'key': 'ranks_types',
   'name': 'Ranks type',
   'ui_group_id': 3},
  {'is_visible': True,
   'key': 'vessel_types_subgroups',
   'name': 'Vessels types subgroups',
   'ui_group_id': 0},
  {'is_visible': True,
   'key': 'agreement_types',
   'name': 'Agreement Types',
   'ui_group_id': 1},
  {'is_visible': True,
   'key': 'vessel_types_groups',
   'name': 'Vessels types groups',
   'ui_group_id': 0},
  {'is_visible

In [84]:
get_dict('phone_types')


[{'id': 1, 'order': 1, 'value': 'Mobile'},
 {'id': 2, 'order': 2, 'value': 'Local'},
 {'id': 3, 'order': 3, 'value': 'Work'}]

In [ ]:
# обработка спарсенных данных и получение необходимой информации
from functions import get_names, extract_date_to_iso, get_emails_list, get_id_raw

#Обработка ФИО
name, middle_name, surname = get_names(main_info['Main info']['Name / Surname:'])

#Обработка даты и места рождения
date_of_birth, place_of_birth = extract_date_to_iso(main_info['Main info']['Birthday / Place of birth:'])

# Добавление емейлов
emails = get_emails_list(main_info['Main info']['E-mail:'])

# добавление gender_id
gender_id = get_id_raw(gender,main_info['Biometrics']['Sex'])



In [98]:
personal_id =[]
[personal_id.append(dct['No.']) if dct['Title of document']=='International passport' else None for dct in main_info['Passports / Smbk']]
personal_id[0]

'FJ 431255'

In [89]:
# проверка работоспособности 
# Будет удалено

import functions
importlib.reload(functions)

for file in html_files:
    # получаем объект soup
    soup = functions.get_html_content(file)
    # парсим всю инфу
    main_info = functions.main_parser(soup)
    pprint(main_info['Passports / Smbk'], sort_dicts=False)#['Phones:'])
    print(functions.get_emails_list(main_info['Main info']['E-mail:']))
    print()

[{'Title of document': "Seaman's book",
  'No.': '',
  'Date of issue': '',
  'Country of issue': '',
  'Valid up': ''},
 {'Title of document': 'International passport',
  'No.': '',
  'Date of issue': '',
  'Country of issue': '',
  'Valid up': ''}]
[{'email': 's-semenenko@bk.ru', 'comment': 'comment', 'uuid': None}]

[{'Title of document': "Seaman's book",
  'No.': '195390',
  'Date of issue': '09.07.2015',
  'Country of issue': 'egypt',
  'Valid up': '30.01.2020'},
 {'Title of document': 'International passport',
  'No.': 'A16081502',
  'Date of issue': '02.09.2015',
  'Country of issue': 'egypt',
  'Valid up': '01.09.2022'}]
[{'email': 'cap.omarmohamed@gmail.com', 'comment': 'comment', 'uuid': None}]

[{'Title of document': "Seaman's book",
  'No.': '******',
  'Date of issue': '19.02.2020',
  'Country of issue': 'Russia',
  'Valid up': '18.02.2025'},
 {'Title of document': 'International passport',
  'No.': '******',
  'Date of issue': '26.03.2020',
  'Country of issue': 'Russia',

In [ ]:
# POST /seafarers/main
 seafarer_dict = {
     # "photo": FILE, 
     "name": name,
     "middle_name": middle_name,
     "surname": surname,
     
    
   
     # "rank_id": INTEGER,
     # "additional_ranks_id": [int, int, int, ..],
     
     # "calling_name": STRING,
     # "available_from": DATE,
     # "available_to": DATE,
     
     "date_of_birth": date_of_birth,
     "place_of_birth": place_of_birth,
     "gender_id": gender_id,
     
     # "marital_status_id": INTEGER,
     # "nationality_country_id": INTEGER,
    
     "emails": emails,
     
     # "resident_status_id": INTEGER,
     "fast_note": notes,
     # "phone_numbers": [
     #     {"uuid":null, "country_id":3, "number":"11223344", "type_id":1, "comment":"comment"}
     # ],
     # "personal_id": INTEGER,
     # "language_id": INTEGER,
 }
 seafarer_dict

In [ ]:

# # ячейка для присвоения значений переменным
# photo = 
# name =
# middle_name =
# surname =

# rank_id = get_id(ranks,extract_anything('out\\out_min\\seafarers_104588.html', key))

# additional_ranks_id =
# calling_name =
# available_from =
# available_to =
# # date_of_birth =
# place_of_birth =
# gender_id =
# marital_status_id =
# nationality_country_id =
# emails =
# resident_status_id =
# fast_note =
# phone_numbers =
# personal_id =
# language_id =

# # Словарь формат json с основной информацией о моряке,
# # который будет отправлен для получения uuid моряка
# # - по нему в дальнейшем будет добавляться информация в БД
# main = {
#     "photo": photo #FILE, ?? 
    
#     "name": name #STRING,
#     "middle_name": middle_name #STRING,
#     "surname": surname #STRING,
#     "rank_id": rank_id #INTEGER,
#     "additional_ranks_id": additional_ranks_id #[int, int, int, ..],
#     "calling_name": calling_name #STRING,
#     "available_from": available_from #DATE,
#     "available_to": available_to #DATE,
#     "date_of_birth": date_of_birth #DATE,
#     "place_of_birth": place_of_birth #STRING,
#     "gender_id":gender_id # INTEGER,
#     "marital_status_id":marital_status_id # INTEGER,
#     "nationality_country_id":nationality_country_id # INTEGER,
#     "emails":emails # [
#         # {"email":"email1@mail.com", "comment":"comment", "uuid":null}
#     # ],
#     "resident_status_id": resident_status_id #INTEGER,
#     "fast_note": fast_note #STRING,  
#     "phone_numbers": phone_numbers #[
#         # {"uuid":null, "country_id":3, "number":"11223344", "type_id":1, "comment":"comment"}
#     # ],
#     "personal_id": personal_id #INTEGER,
#     "language_id": language_id #INTEGER,
# }

In [ ]:
# # определение url ссылок для использования

# # основной домен
# domain = 'https://staffdev.crew-man.com/api/v1/'

# # Добавление основной инфы моряка:POST
# main_info = 'seafarers/main'

# # seafarer_uuid = response_main.text
# # Добавление адресов моряка:POST
# # adress = f'seafarers/{seafarer_uuid}/addresses'

# # Добавление сертификатов (документов) моряка:POST
# # certificates = f'seafarers/{seafarer_uuid}/certificates'

# # Добавление родственников (kins) моряка:POST
# # relatives = f'seafarers/{seafarer_uuid}/relatives'

# # Добавление sea service моряка:POST
# contracts =  'contracts'

# route = main_info
# url = domain + route


In [ ]:
# from bs4 import MarkupResemblesLocatorWarning
# import warnings
# warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

In [ ]:
# post ={
#     "photo": FILE, ??
#     "name": NAME,
#     "middle_name": STRING,
#     "surname": SURNAME,
#     "rank_id": INTEGER,
#     # "additional_ranks_id": [int, int, int, ..],
#     "calling_name": STRING,
#     "available_from": DATE,
#     "available_to": DATE,
#     "date_of_birth": DATE_OF_BIRTH,
#     "place_of_birth": STRING,
#     "gender_id": INTEGER,
#     "marital_status_id": INTEGER,
#     "nationality_country_id": INTEGER,
#     "emails": [
#         {"email":"email1@mail.com", "comment":"comment", "uuid":null}
#     ],
#     "resident_status_id": INTEGER,
#     "fast_note": STRING,
#     "phone_numbers": [
#         {"uuid":null, "country_id":3, "number":"11223344", "type_id":1, "comment":"comment"}
#     ],
#     "personal_id": INTEGER,
#     "language_id": INTEGER,
# }

In [ ]:
# main = {
    
#     "name": [name.strip() for name in 'Name / Surname:'.split('/')][0],
    
#     "surname": [name.strip() for name in 'Name / Surname:'.split('/')][1][:-1],
    
#     "rank_id": INTEGER, я так понимаю это ид должностей где они беруться?
#     "additional_ranks_id": [int, int, int, ..],
    
    
#     "date_of_birth": DATE,
#     # "place_of_birth": STRING,
    
#     "emails": [
#         {"email":"email1@mail.com", "comment":"comment", "uuid":null}
#     ],
#     "resident_status_id": INTEGER, резидент РФ?
#     "fast_note": STRING,
#     "phone_numbers": [
#         {"uuid":null, "country_id":3, "number":"11223344", "type_id":1, "comment":"comment"}
#     ],
#     "personal_id": INTEGER, где берется?
#     "language_id": INTEGER, где берется?
# }